In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('./data/preprocessed_Bank_Customer_Churn_Prediction.csv')

In [3]:
df


,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,0,1,42,2,0.00,1,1,1,101348.88,1
1,608,2,1,41,1,83807.86,1,0,1,112542.58,0
2,502,0,1,42,8,159660.80,3,1,0,113931.57,1
3,699,0,1,39,1,0.00,2,0,0,93826.63,0
4,850,2,1,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,0,39,5,0.00,2,1,0,96270.64,0
9996,516,0,0,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,1,36,7,0.00,1,0,1,42085.58,1
9998,772,1,0,42,3,75075.31,2,1,0,92888.52,1


### XGBOOST 사용

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# XGBoost 모델 생성 및 학습
model = xgb.XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss', 
    use_label_encoder=False, 
    random_state=42
)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Accuracy: 0.8470
Classification Report:
               precision    recall  f1-score   support

           0     0.8730    0.9454    0.9078      1593
           1     0.6836    0.4619    0.5513       407

    accuracy                         0.8470      2000
   macro avg     0.7783    0.7037    0.7295      2000
weighted avg     0.8345    0.8470    0.8352      2000



c:\Users\Playdata\AppData\Local\anaconda3\envs\psystudy_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:46:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [10]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report

# 데이터 준비
y = df['churn']
X = df.drop(columns=['churn'])

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 하이퍼파라미터 설정
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': 42,
    'learning_rate': 0.1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 100
}

# ✅ XGBoost의 cv 기능 사용
cv_results = xgb.cv(
    params=params,
    dtrain=xgb.DMatrix(X_train, label=y_train),
    num_boost_round=100,
    nfold=5,
    stratified=True,
    metrics='logloss',
    early_stopping_rounds=10
)

# 최적 n_estimators 설정
best_n_estimators = len(cv_results)
params['n_estimators'] = best_n_estimators

# 최적 모델 학습
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f'Best n_estimators: {best_n_estimators}')
print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


c:\Users\Playdata\AppData\Local\anaconda3\envs\psystudy_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [18:50:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Best n_estimators: 79
Accuracy: 0.8740
Classification Report:
               precision    recall  f1-score   support

           0     0.8834    0.9699    0.9246      1593
           1     0.8088    0.4988    0.6170       407

    accuracy                         0.8740      2000
   macro avg     0.8461    0.7343    0.7708      2000
weighted avg     0.8682    0.8740    0.8620      2000



### LGB사용

In [6]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)  # 검증 데이터 설정

# LightGBM 모델 학습
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': -1
}

# callbacks을 사용하여 early stopping 적용
model = lgb.train(params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], valid_names=['valid'],
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 예측
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)


print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Training until validation scores don't improve for 10 rounds
[10]	valid's binary_logloss: 0.370556
[20]	valid's binary_logloss: 0.344806
[30]	valid's binary_logloss: 0.338654
[40]	valid's binary_logloss: 0.335742
[50]	valid's binary_logloss: 0.335947
Early stopping, best iteration is:
[45]	valid's binary_logloss: 0.335173
Accuracy: 0.8645
Classification Report:
               precision    recall  f1-score   support

           0     0.8781    0.9636    0.9189      1593
           1     0.7698    0.4767    0.5888       407

    accuracy                         0.8645      2000
   macro avg     0.8240    0.7201    0.7538      2000
weighted avg     0.8561    0.8645    0.8517      2000



### LGB gridsearch사용

In [7]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# LightGBM 모델 객체 생성
model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', verbosity=-1)

# 하이퍼파라미터 검색 공간 정의
param_grid = {
    'num_leaves': [20, 31, 50],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_samples': [10, 20, 50],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Grid Search 수행
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # 3-폴드 교차검증
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", grid_search.best_params_)

# 최적 하이퍼파라미터 적용하여 모델 재학습
best_model = grid_search.best_estimator_

# 예측
y_pred = best_model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred,digits=4)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Fitting 3 folds for each of 729 candidates, totalling 2187 fits
Best parameters found: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_samples': 50, 'num_leaves': 50, 'subsample': 0.7}
Accuracy: 0.8640
Classification Report:
               precision    recall  f1-score   support

           0     0.8746    0.9680    0.9190      1593
           1     0.7848    0.4570    0.5776       407

    accuracy                         0.8640      2000
   macro avg     0.8297    0.7125    0.7483      2000
weighted avg     0.8564    0.8640    0.8495      2000



---

# optuna 사용


In [8]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    model = lgb.train(params, train_data, num_boost_round=100,
                      valid_sets=[valid_data], 
                      callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])
    
    y_pred = model.predict(X_test)
    y_pred = [1 if x > 0.5 else 0 for x in y_pred]
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", study.best_params)

# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_params
best_params.update({'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1})

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(best_params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], 
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 최적 모델 예측
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


[I 2025-02-08 18:50:19,607] A new study created in memory with name: no-name-6bd9a99b-e3d9-4c90-b21b-5ef0ac5ab335
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases

Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.404552
[20]	valid_0's binary_logloss: 0.366253
[30]	valid_0's binary_logloss: 0.350865
[40]	valid_0's binary_logloss: 0.342715
[50]	valid_0's binary_logloss: 0.340199
[60]	valid_0's binary_logloss: 0.338635
[70]	valid_0's binary_logloss: 0.337512
[80]	valid_0's binary_logloss: 0.33622
[90]	valid_0's binary_logloss: 0.336577
Early stopping, best iteration is:
[83]	valid_0's binary_logloss: 0.33605
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.402387
[20]	valid_0's binary_logloss: 0.363613
[30]	valid_0's binary_logloss: 0.34958
[40]	valid_0's binary_logloss: 0.342296


[I 2025-02-08 18:50:19,927] Trial 1 finished with value: 0.869 and parameters: {'num_leaves': 89, 'max_depth': 7, 'learning_rate': 0.08473123855018388, 'min_child_samples': 64, 'subsample': 0.84723845068526, 'colsample_bytree': 0.5892572070783184}. Best is trial 1 with value: 0.869.


[50]	valid_0's binary_logloss: 0.339905
[60]	valid_0's binary_logloss: 0.339954
[70]	valid_0's binary_logloss: 0.339122
[80]	valid_0's binary_logloss: 0.33879
Early stopping, best iteration is:
[78]	valid_0's binary_logloss: 0.338658
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.457561
[20]	valid_0's binary_logloss: 0.424294
[30]	valid_0's binary_logloss: 0.405224
[40]	valid_0's binary_logloss: 0.388149
[50]	valid_0's binary_logloss: 0.377791
[60]	valid_0's binary_logloss: 0.367672
[70]	valid_0's binary_logloss: 0.362334
[80]	valid_0's binary_logloss: 0.356074


C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)


...

Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.368994
[20]	valid_0's binary_logloss: 0.343718
[30]	valid_0's binary_logloss: 0.341263
[40]	valid_0's binary_logloss: 0.34258
Early stopping, best iteration is:
[33]	valid_0's binary_logloss: 0.340092
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.437442
[20]	valid_0's binary_logloss: 0.401263
[30]	valid_0's binary_logloss: 0.381622
[40]	valid_0's binary_logloss: 0.365963
[50]	valid_0's binary_logloss: 0.357502
[60]	valid_0's binary_logloss: 0.351158
[70]	valid_0's binary_logloss: 0.346712
[80]	valid_0's binary_logloss: 0.343185


[I 2025-02-08 18:50:24,337] Trial 27 finished with value: 0.865 and parameters: {'num_leaves': 94, 'max_depth': 4, 'learning_rate': 0.048108854877583596, 'min_child_samples': 73, 'subsample': 0.5591750727139418, 'colsample_bytree': 0.5978180679522447}. Best is trial 1 with value: 0.869.
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykern

[90]	valid_0's binary_logloss: 0.341405
[100]	valid_0's binary_logloss: 0.340279
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.340279
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.396022
[20]	valid_0's binary_logloss: 0.358751
[30]	valid_0's binary_logloss: 0.345571
[40]	valid_0's binary_logloss: 0.339923
[50]	valid_0's binary_logloss: 0.338891
[60]	valid_0's binary_logloss: 0.33844
Early stopping, best iteration is:
[53]	valid_0's binary_logloss: 0.338076


C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_16476\2289443087.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)


Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.460388
[20]	valid_0's binary_logloss: 0.428654
[30]	valid_0's binary_logloss: 0.409458
[40]	valid_0's binary_logloss: 0.392537
[50]	valid_0's binary_logloss: 0.381591
[60]	valid_0's binary_logloss: 0.371906
[70]	valid_0's binary_logloss: 0.366533
[80]	valid_0's binary_logloss: 0.35975
[90]	valid_0's binary_logloss: 0.354762
[100]	valid_0's binary_logloss: 0.351591
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.351591
Best parameters found: {'num_leaves': 89, 'max_depth': 7, 'learning_rate': 0.08473123855018388, 'min_child_samples': 64, 'subsample': 0.84723845068526, 'colsample_bytree': 0.5892572070783184}
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.402387
[20]	valid_0's binary_logloss: 0.363613
[30]	valid_0's binary_logloss: 0.34958
[40]	valid_0's binary_logloss: 0.342296
[50]	valid_0's binary_logloss: 0.33990

In [9]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    model = lgb.train(params, train_data, num_boost_round=100,
                      valid_sets=[valid_data], 
                      callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])
    
    y_pred_proba = model.predict(X_test)
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    best_threshold = thresholds[(precision + recall).argmax()]  # 최적의 threshold 선택
    
    y_pred = [1 if x > best_threshold else 0 for x in y_pred_proba]
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", study.best_params)

# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_params
best_params.update({'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1})

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(best_params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], 
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 최적 모델 예측
y_pred_proba = model.predict(X_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
best_threshold = thresholds[(precision + recall).argmax()]  # 최적 threshold 선택

print(f'Best Threshold: {best_threshold:.4f}')

y_pred = [1 if x > best_threshold else 0 for x in y_pred_proba]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


[I 2025-02-08 18:50:24,860] A new study created in memory with name: no-name-781d9814-8eaa-4052-89ba-a843e3f69abd


Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.374636
[20]	valid_0's binary_logloss: 0.345134
[30]	valid_0's binary_logloss: 0.341209


[I 2025-02-08 18:50:24,987] Trial 0 finished with value: 0.865 and parameters: {'num_leaves': 112, 'max_depth': 14, 'learning_rate': 0.14285428959672658, 'min_child_samples': 98, 'subsample': 0.7676272782375093, 'colsample_bytree': 0.6217725186575114}. Best is trial 0 with value: 0.865.


[40]	valid_0's binary_logloss: 0.342151
Early stopping, best iteration is:
[33]	valid_0's binary_logloss: 0.340064
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.368927
[20]	valid_0's binary_logloss: 0.342516
[30]	valid_0's binary_logloss: 0.336167
[40]	valid_0's binary_logloss: 0.332948
[50]	valid_0's binary_logloss: 0.332025
[60]	valid_0's binary_logloss: 0.332129
Early stopping, best iteration is:
[55]	valid_0's binary_logloss: 0.33168


[I 2025-02-08 18:50:25,070] Trial 1 finished with value: 0.864 and parameters: {'num_leaves': 141, 'max_depth': 5, 'learning_rate': 0.12503668910271215, 'min_child_samples': 6, 'subsample': 0.8596630273119743, 'colsample_bytree': 0.8467538379348268}. Best is trial 0 with value: 0.865.
[I 2025-02-08 18:50:25,131] Trial 2 finished with value: 0.8665 and parameters: {'num_leaves': 53, 'max_depth': 10, 'learning_rate': 0.1478767621133668, 'min_child_samples': 56, 'subsample': 0.6721911360409354, 'colsample_bytree': 0.8610303946510053}. Best is trial 2 with value: 0.8665.


...

[60]	valid_0's binary_logloss: 0.340722
[70]	valid_0's binary_logloss: 0.339278
[80]	valid_0's binary_logloss: 0.337719
[90]	valid_0's binary_logloss: 0.336296
[100]	valid_0's binary_logloss: 0.336159
Did not meet early stopping. Best iteration is:
[99]	valid_0's binary_logloss: 0.336052
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.453253
[20]	valid_0's binary_logloss: 0.419425
[30]	valid_0's binary_logloss: 0.399355
[40]	valid_0's binary_logloss: 0.382902
[50]	valid_0's binary_logloss: 0.372464
[60]	valid_0's binary_logloss: 0.363057
[70]	valid_0's binary_logloss: 0.35777
[80]	valid_0's binary_logloss: 0.351942
[90]	valid_0's binary_logloss: 0.347559
[100]	valid_0's binary_logloss: 0.345099
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.345099


[I 2025-02-08 18:50:28,543] Trial 27 finished with value: 0.8195 and parameters: {'num_leaves': 42, 'max_depth': 5, 'learning_rate': 0.0733212669581086, 'min_child_samples': 35, 'subsample': 0.9981095854220816, 'colsample_bytree': 0.6374861212499175}. Best is trial 6 with value: 0.8705.


Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.411642
[20]	valid_0's binary_logloss: 0.373065
[30]	valid_0's binary_logloss: 0.356103
[40]	valid_0's binary_logloss: 0.345166
[50]	valid_0's binary_logloss: 0.340816
[60]	valid_0's binary_logloss: 0.337591
[70]	valid_0's binary_logloss: 0.336328
[80]	valid_0's binary_logloss: 0.335181
[90]	valid_0's binary_logloss: 0.334202
Early stopping, best iteration is:
[89]	valid_0's binary_logloss: 0.334098
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.470183
[20]	valid_0's binary_logloss: 0.446247
[30]	valid_0's binary_logloss: 0.428242
[40]	valid_0's binary_logloss: 0.413347
[50]	valid_0's binary_logloss: 0.401769
[60]	valid_0's binary_logloss: 0.392368
[70]	valid_0's binary_logloss: 0.384858
[80]	valid_0's binary_logloss: 0.378726
[90]	valid_0's binary_logloss: 0.373686
[100]	valid_0's binary_logloss: 0.369296
Did not meet early stopping. Best iteratio

[I 2025-02-08 18:50:28,664] Trial 28 finished with value: 0.8615 and parameters: {'num_leaves': 115, 'max_depth': 4, 'learning_rate': 0.014335735025662648, 'min_child_samples': 22, 'subsample': 0.5584230818236376, 'colsample_bytree': 0.9744881319439486}. Best is trial 6 with value: 0.8705.
[I 2025-02-08 18:50:28,762] Trial 29 finished with value: 0.813 and parameters: {'num_leaves': 76, 'max_depth': 6, 'learning_rate': 0.13536566303400632, 'min_child_samples': 90, 'subsample': 0.7802246671795031, 'colsample_bytree': 0.5954340176720967}. Best is trial 6 with value: 0.8705.


Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.377209
[20]	valid_0's binary_logloss: 0.347396
[30]	valid_0's binary_logloss: 0.340755
[40]	valid_0's binary_logloss: 0.338715
Early stopping, best iteration is:
[35]	valid_0's binary_logloss: 0.338588
Best parameters found: {'num_leaves': 137, 'max_depth': 4, 'learning_rate': 0.23980918805110849, 'min_child_samples': 89, 'subsample': 0.5540157845788954, 'colsample_bytree': 0.5181621140857728}
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.358105
[20]	valid_0's binary_logloss: 0.338639
[30]	valid_0's binary_logloss: 0.334886
[40]	valid_0's binary_logloss: 0.333847
Early stopping, best iteration is:
[36]	valid_0's binary_logloss: 0.332717
Best Threshold: 0.5387
Accuracy: 0.8705
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.98      0.92      1593
           1       0.83      0.45      0

Precision과 Recall 균형 조정

Precision 증가(0.80 → 0.84)
Recall 감소(0.48 → 0.44)
이탈 고객을 보다 정확하게 예측하지만, 일부를 놓칠 가능성이 있음
비즈니스 목표에 따라 Threshold를 조정할 필요 있음!
→ 이탈 고객 Recall을 높이고 싶다면 더 낮은 Threshold를 시도하는 것도 가능